In [27]:
cd drive/MyDrive/Colab\ Notebooks/Trimmed_TC_updated


[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Trimmed_TC_updated'
/content/drive/MyDrive/Colab Notebooks/Trimmed_TC_updated


In [28]:
ls

 combined_data.csv
'h2_flux88_abs0_mr_surf0_571s - Sheet1.csv'
'h2_flux88_abs0_surf0_431s - Sheet1.csv'
'h2_flux88_abs0_surf0_585s - Sheet2.csv'
'h2_flux88_abs0_surf0_probeUp_590s - Sheet2.csv'
'h2_flux88_abs0_wr_surf0_368s - Sheet1.csv'
'h2_flux88_abs25_newSalt_surf0_172s - Sheet1.csv'
'h2_flux88_abs25_newSalt_wr_surf0_123s - Sheet1.csv'
'h2_flux88_abs25_surf0_493s - Sheet1.csv'
'h2_flux88_abs25_wr_surf0_393s - Sheet1.csv'
'h2_flux88_abs25_wr_surfParAdded_169s - Sheet1.csv'
'h2_flux88_abs25_wr_surfSimD_525s - Sheet1.csv'
'h2_flux88_abs92_surf0_115s - Sheet1.csv'
'h2_flux88_abs92_surf0_probeUp_193s - Sheet1.csv'
'h2_flux88_abs92_wr_surf0_215s - Sheet1.csv'
 h3_flux88_abs0_mr_surf0_635s-Sheet1.csv
 h3_flux88_abs0_surf0_644s-Sheet2.csv
'h3_flux88_abs0_surf0_719s - Sheet1.csv'
 h3_flux88_abs0_surf0_825s-Sheet3.csv
 h3_flux88_abs0_wr_surf0_416s-Sheet1.csv
 h3_flux88_abs25_mr_surf0_796s-Sheet1.csv
 h3_flux88_abs25_surf0_439s-Sheet2.csv
 h3_flux88_abs25_surf0_660s-Sheet1.csv
 h3_flux88_abs25

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Combining All Data Files into One

In [30]:
import os
import pandas as pd
import re

pattern = r"h(\d+)_flux(\d+)_abs(\d+).*?_surf(\d+).*?_(\d+)s"

# % mapping tables for quick lookup
h_map = {2:0.0375, 3:0.084, 6:0.1575}
flux_map = {88: 25900, 78: 21250, 73: 19400}
abs_val_map = {0: 3, 92: 100}
surf_map = {0:0.98, 1:0.76}

dataframes = []
for filename in os.listdir("./"):
    if filename.endswith(".csv"):

        match = re.search(pattern, filename)
        if match:
            h = int(match.group(1))
            flux = int(match.group(2))
            abs_val = int(match.group(3))
            surf = int(match.group(4))
            min_time = int(match.group(5))

            # flux from 73 → 19.4, 88 → 25.9
            if h in h_map:
              h = h_map[h]
            if flux in flux_map:
              flux = flux_map[flux]

            if abs_val in abs_val_map:
              abs_val = abs_val_map[abs_val]

            if surf in surf_map:
              surf = surf_map[surf]



            print(filename)
            file_path = os.path.join("", filename)
            print(file_path)
            data = pd.read_csv(file_path, encoding="utf-8-sig")  # UTF-8 with BOM

            if 'Time' in data.columns:
                data = data[data['Time'] >= min_time].copy()

            data["h"] = h
            data["flux"] = flux
            data["abs"] = abs_val
            data["surf"] = surf

            dataframes.append(data)

# Combine all dataframes into one
combined_data = pd.concat(dataframes, ignore_index=True)

print(combined_data.head())

combined_data.to_csv("combined_data.csv", index=False)


h6_flux88_abs20_wr_surf0_734s - Sheet1.csv
h6_flux88_abs20_wr_surf0_734s - Sheet1.csv
h2_flux88_abs25_surf0_493s - Sheet1.csv
h2_flux88_abs25_surf0_493s - Sheet1.csv
h6_flux78_abs0_surf0_newSalt_641s - Sheet1.csv
h6_flux78_abs0_surf0_newSalt_641s - Sheet1.csv
h6_flux88_abs0_surf1_newSalt_573s - Sheet2.csv
h6_flux88_abs0_surf1_newSalt_573s - Sheet2.csv
h2_flux88_abs92_surf0_115s - Sheet1.csv
h2_flux88_abs92_surf0_115s - Sheet1.csv
h6_flux88_abs20_surf1_675s - Sheet3.csv
h6_flux88_abs20_surf1_675s - Sheet3.csv
h6_flux88_abs0_surf1_790s - Sheet1.csv
h6_flux88_abs0_surf1_790s - Sheet1.csv
h6_flux88_abs92_surf0_longRun_647s - Sheet2.csv
h6_flux88_abs92_surf0_longRun_647s - Sheet2.csv
h2_flux88_abs0_surf0_431s - Sheet1.csv
h2_flux88_abs0_surf0_431s - Sheet1.csv
h6_flux88_abs20_surf0_781s - Sheet2.csv
h6_flux88_abs20_surf0_781s - Sheet2.csv
h6_flux73_abs0_surf1_660s - Sheet1.csv
h6_flux73_abs0_surf1_660s - Sheet1.csv
h6_flux88_abs20_surf1_830s - Sheet2.csv
h6_flux88_abs20_surf1_830s - Sheet2.

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
# so for this model, the training data will be
# h (depth), flux (q0), abs (absorption coefficient k), surf (surface emisisivty epsilon), and varying time (tStep)
# and y will be the thermal couples values FOR THAT TIME
data = pd.read_csv("combined_data.csv")

# removing these two columns, some data files have these, some don't. I'll worry about them later +
# need to ask what they do again
data.drop(columns=["TC_9.5", "TC_Bottom_rec_groove","TC_wall_ins_ext", "TC_bottom_ins_groove"], axis=1, inplace=True)

# making sure no nulls in  other  rows after r emoving these two
print("Null Check: ", data.isnull().sum())

X = data[["Time", "h", "flux", "abs", "surf"]]
print("INPUTS: \n", X.head())
y = data.drop(columns=["Time", "h", "flux", "abs", "surf"])
print("OUPUTS: \n", y.head())

y_columns = y.columns
# MinMaxScaler -> normalization, standardize when data is not normal (not in our case)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)




input_size = X_train.shape[1]
print("input size: ", input_size)

output_size = y_train.shape[1]
print("output size: ", output_size)

print("Amount of Training Data: ", X_train.shape[0])






Null Check:  Time       0
TC1_tip    0
TC2        0
TC3        0
TC4        0
TC5        0
TC6        0
TC7        0
TC8        0
TC9        0
TC10       0
h          0
flux       0
abs        0
surf       0
dtype: int64
INPUTS: 
    Time       h   flux  abs  surf
0   734  0.1575  25900   20  0.98
1   735  0.1575  25900   20  0.98
2   736  0.1575  25900   20  0.98
3   737  0.1575  25900   20  0.98
4   738  0.1575  25900   20  0.98
OUPUTS: 
    TC1_tip     TC2     TC3     TC4     TC5     TC6     TC7     TC8     TC9  \
0   360.74  360.08  360.53  360.79  360.14  361.79  358.63  359.26  356.22   
1   360.73  360.05  360.45  360.77  360.10  361.77  358.58  359.25  356.10   
2   360.73  360.01  360.37  360.76  360.05  361.74  358.51  359.34  356.20   
3   360.71  359.99  360.33  360.75  360.02  361.73  358.51  359.47  356.41   
4   360.69  359.95  360.32  360.76  360.01  361.73  358.55  359.68  356.62   

     TC10  
0  335.77  
1  335.32  
2  334.95  
3  334.62  
4  334.35  
input size:  5

In [32]:
# basic neural net
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = NeuralNet(input_size, output_size)

# coommon loss func n optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 500
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad() # resetting gradients
    outputs = model(X_train)

    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}")

# no more calculating grad descent/back propgation, only forward
model.eval()
with torch.no_grad():
    y_pred = model(X_test).numpy()

# change back predictions and true values
y_pred_real = y_scaler.inverse_transform(y_pred)
y_test_real = y_scaler.inverse_transform(y_test.numpy())

# get RMSE per output
rmse_per_output = np.sqrt(np.mean((y_pred_real - y_test_real)**2, axis=0))
rmse_overall = np.mean(rmse_per_output)


print("\nRMSE per output (°C):")
for col, rmse in zip(y_columns, rmse_per_output):
    print(f"{col}: {rmse:.3f} °C")

print(f"\nAverage RMSE across all outputs: {rmse_overall:.3f} °C")

torch.save(model.state_dict(), "thermal_model_weights.pth")
print("Model weights saved to 'thermal_model_weights.pth'")

Epoch [50/500], Loss: 0.04046384245157242
Epoch [100/500], Loss: 0.02190983295440674
Epoch [150/500], Loss: 0.012881455011665821
Epoch [200/500], Loss: 0.009649925865232944
Epoch [250/500], Loss: 0.008036956191062927
Epoch [300/500], Loss: 0.007689033634960651
Epoch [350/500], Loss: 0.007546356879174709
Epoch [400/500], Loss: 0.007389561738818884
Epoch [450/500], Loss: 0.007201268337666988
Epoch [500/500], Loss: 0.007025961298495531

RMSE per output (°C):
TC1_tip: 32.332 °C
TC2: 31.944 °C
TC3: 32.295 °C
TC4: 31.559 °C
TC5: 31.399 °C
TC6: 31.322 °C
TC7: 31.868 °C
TC8: 32.751 °C
TC9: 35.395 °C
TC10: 37.864 °C

Average RMSE across all outputs: 32.873 °C
Model weights saved to 'thermal_model_weights.pth'


# Using a Trained Model
To use a trained NN, you need the weights as a .pt or .pth file AND the architecture of the NN

In [33]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


input_size = 5
output_size = y.shape[1]
model = NeuralNet(input_size, output_size)
model.load_state_dict(torch.load('thermal_model_weights.pth')) # load  weights
model.eval() # eval mode, no trianing


NeuralNet(
  (fc1): Linear(in_features=5, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [34]:
# time (tStep), h (y), flux (q0), abs (k), surf (epsilon),
new_data = [[1049.0, 0.1575, 25900, 3, 0.98]]
# still have to scale d ata
new_data = X_scaler.transform(new_data)
new_data_tensor = torch.tensor(new_data, dtype=torch.float32)



with torch.no_grad():
    predictions = model(new_data_tensor)

# reverse normalziation to see the acutal outputs
predictions = y_scaler.inverse_transform(predictions.numpy())
print("Predictions:", predictions)

Predictions: [[349.1187  348.43625 352.2031  354.99475 353.1865  351.7591  352.80432
  350.80362 349.60376 334.27243]]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
